In [4]:
# allow for imports from ../src directory

from os import path as os_path, getcwd
from sys import path as sys_path

sys_path.append(os_path.dirname(getcwd())) 
#sys.path.append(f"{os_path.dirname(getcwd())}/src")

In [7]:
from src.modelling import set_fluid_parameters

In [80]:
import numpy as np
from pathlib import Path

In [68]:
fluid_parameters = dict({"kcl": dict({"name": "Potassium Chloride", "component": ["K", "Cl"], "valence": np.array([
    1.0, -1.0]), "charge": None, "temperature": 1075.0, "concentration": np.array([19.5, 19.5]),
    "epsilon_r": 1.0, "index": 1, "charge_pair": None, "rho": None, "beta": 0.0, "epsilon": 0.0})})


fluid_parameters["lj1"] = dict({"name": "Lennard-Jones liquid (1-comp)", "component": ["LJ"], "valence": np.array([0]),
                                "charge": None, "temperature": 298.15, "concentration": np.array([0.5]), "epsilon_r": 1, "index": 2,
                                "charge_pair": None, "rho": None, "beta": 0.0, "epsilon": 0.0})


fluid_parameters["lj2"] = dict({"name": "Lennard-Jones liquid (2-comp)", "component": ["A1", "A2"], "valence": np.array([0.0, 0.0]),
                                "charge": None, "temperature": 298.15, "concentration": np.array([0.5, 0.5]), "epsilon_r": 1, "index": 5,
                                "charge_pair": None, "rho": None, "beta": 0.0, "epsilon": 0.0})


fluid_parameters["h2o"] = dict({"name": "Liquid water", "component": ["H", "2O"], "valence": np.array([
    1.0, -1.0]), "temperature": 298.0, "concentration": np.array([1.0, 1.0]),
    "epsilon_r": 1.0, "index": 2})


fluid_parameters["2_2"] = dict({"name": "2-2 Aqueous electrolyte", "component": ["+2", "-2"], "valence": np.array([
    2.0, -2.0]), "temperature": 298.0, "concentration": np.array([1.0, 1.0]),
    "epsilon_r": 78.3, "index": 3})


In [69]:
set_fluid_parameters("lj1")

Fluid(name='Lennard-Jones liquid (1-comp)', symbol='LJ', component=['LJ'], valence=array([0.]), charge=array([0]), temperature=298.15, concentration=array([1.]), epsilon_r=1, n_component=1, n_pair=1, index=2, charge_pair=array([0.]), rho=array([0.]), beta=0.0, epsilon=0.0, cr_path='')

In [70]:
set_fluid_parameters("lj2")

Fluid(name='Lennard-Jones liquid (2-comp)', symbol='A1A2', component=['A1', 'A2'], valence=array([0., 0.]), charge=array([0]), temperature=298.15, concentration=array([0.5, 0.5]), epsilon_r=1, n_component=2, n_pair=3, index=5, charge_pair=array([0.]), rho=array([0.]), beta=0.0, epsilon=0.0, cr_path='')

In [71]:
import json
from json import JSONEncoder

In [84]:
class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)
        
def dict_to_json(d, json_filename, n_indent=4):
    with open(json_filename, 'w', encoding='utf-8') as f:
        #json.dump(data, f, ensure_ascii=False, indent=4)
        json.dump(d, f, indent = n_indent, ensure_ascii=False, cls=NumpyArrayEncoder) 


In [85]:
json_filename = Path.cwd().parent / "data/fluid_parameters.json"

In [87]:
dict_to_json(fluid_parameters, json_filename)

In [74]:
!ls

bulk_fluid_explore.ipynb data.toml                singlet_kcl.ipynb
check_units.ipynb        fluid_parameters.ipynb
data.json                numerics.ipynb


In [75]:
# from types import NoneType

In [76]:


def _dumps_value(value):
    if isinstance(value, bool):
        return "true" if value else "false"
    elif isinstance(value, (int, float)):
        return str(value)
    elif isinstance(value, str):
        return f'"{value}"'
    elif isinstance(value, list):
        return f"[{', '.join(_dumps_value(v) for v in value)}]"
    elif isinstance(value, np.ndarray):
        return f"[{', '.join(_dumps_value(v) for v in value.tolist())}]"
    elif isinstance(value, type(None)):
        return "nan"
    else:
        raise TypeError(f"{type(value).__name__} {value!r} is not supported")

In [79]:
def dict_to_toml(toml_dict, toml_filename, table=""):
    def tables_at_end(item):
        _, value = item
        return isinstance(value, dict)

    toml = []
    for key, value in sorted(toml_dict.items(), key=tables_at_end):
        if isinstance(value, dict):
            table_key = f"{table}.{key}" if table else key
            toml.append(f"\n[{table_key}]\n{dumps(value, table_key)}")
        else:
            toml.append(f"{key} = {_dumps_value(value)}")
    toml_str = "\n".join(toml)
    with open(toml_filename, 'w', encoding='utf-8') as f:
        f.write(toml_str)
    return toml_str

In [83]:
toml_filename = Path.cwd().parent / "data/fluid_parameters.toml"

toml_str = dict_to_toml(fluid_parameters, toml_filename=toml_filename)